In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
weather = pd.read_csv(r"C:\Users\user\OneDrive\Documents\Data Analysis Course\Machine Learning\Climate Wins\2. Data Sets\Dataset-weather-prediction-dataset-processed.csv")
answers = pd.read_csv(r"C:\Users\user\OneDrive\Documents\Data Analysis Course\Machine Learning\Climate Wins\2. Data Sets\Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv")

In [3]:
weather = weather.drop(columns=[c for c in ["DATE","MONTH"] if c in weather.columns], errors="ignore")
answers = answers.drop(columns=["DATE"], errors="ignore")

In [4]:
DROP_STATIONS = ["GDANSK", "ROMA", "TOURS"]
weather = weather[[c for c in weather.columns if not any(s in c for s in DROP_STATIONS)]]
answers = answers[[c for c in answers.columns if not any(s in c for s in DROP_STATIONS)]]

In [5]:
import re

_drop_pat = re.compile(r'^[A-Z]+_(wind_speed|snow_depth)$')

def drop_wind_snow(weather):
    to_drop = [c for c in weather.columns if _drop_pat.match(c)]
    print(f"Dropping {len(to_drop)} cols:", to_drop)
    return weather.drop(columns=to_drop, errors="ignore")

weather = drop_wind_snow(weather)
answers= drop_wind_snow(answers)

Dropping 15 cols: ['BASEL_wind_speed', 'BASEL_snow_depth', 'DEBILT_wind_speed', 'DUSSELDORF_wind_speed', 'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth', 'KASSEL_wind_speed', 'LJUBLJANA_wind_speed', 'MAASTRICHT_wind_speed', 'MADRID_wind_speed', 'MUNCHENB_snow_depth', 'OSLO_wind_speed', 'OSLO_snow_depth', 'SONNBLICK_wind_speed', 'VALENTIA_snow_depth']
Dropping 0 cols: []


In [6]:
# Create/fill the three specific station–observation columns from donors
# Receivers -> Donors
specs = [
    ("KASSEL",    "cloud_cover", "LJUBLJANA"),
    ("MUNCHENB",  "pressure",    "OSLO"),
    ("STOCKHOLM", "humidity",    "VALENTIA"),
]

for recv, obs, donor in specs:
    recv_col  = f"{recv}_{obs}"
    donor_col = f"{donor}_{obs}"

    if donor_col not in weather.columns:
        print(f" Donor missing: {donor_col}. Skipping {recv_col}.")
        continue

    if recv_col not in weather.columns:
        # Receiver column absent → create it by copying donor values
        weather[recv_col] = weather[donor_col]
        print(f" Created {recv_col} from {donor_col} (full copy).")
    else:
        # Receiver exists → only fill NaNs from donor
        before = weather[recv_col].isna().sum()
        mask = weather[recv_col].isna() & weather[donor_col].notna()
        weather.loc[mask, recv_col] = weather.loc[mask, donor_col]
        after = weather[recv_col].isna().sum()
        print(f" {recv_col}: filled {before - after} NaNs (now {after} NaNs).")


 Created KASSEL_cloud_cover from LJUBLJANA_cloud_cover (full copy).
 Created MUNCHENB_pressure from OSLO_pressure (full copy).
 Created STOCKHOLM_humidity from VALENTIA_humidity (full copy).


In [7]:
# rebuild the station_summary pivot to reflect the fills
long_weather = (
    weather.melt(var_name="station_obs", value_name="value")
           .dropna(subset=["value"])
)
long_weather[["station","observation"]] = long_weather["station_obs"].str.split("_", n=1, expand=True)
station_summary = long_weather.groupby(["station","observation"])["value"].sum().unstack("observation")

# sanity check the three previously-missing cells
print(station_summary.loc[["KASSEL","MUNCHENB","STOCKHOLM"], ["cloud_cover","pressure","humidity"]])

observation  cloud_cover    pressure  humidity
station                                       
KASSEL          117257.0  23319.5228  17698.72
MUNCHENB        119970.0  23216.2717  17507.11
STOCKHOLM       118668.0  23213.7676  18952.67


In [8]:
weather.shape

(22950, 135)

In [9]:
weather.to_csv("weather_cleaned_filled.csv", index=False)
station_summary.to_csv("station_summary_after_fill.csv")

In [10]:
answers.shape

(22950, 15)

** CNN MODEL

Reason: 

-excellent at finding spatial/local patterns 

-can handle structured tabular input reshaped into grids 

-training is faster and more stable than RNNs

CNN was chosen due to their efficiency at capturing local temporal patterns across multiple time steps, making them suitable for fixed-window weather sequence analysis.

In [11]:
tf.random.set_seed(42)

In [12]:
X=weather
y=answers

In [13]:
class_day = {
    0: 'UNPLEASANT',
    1: 'PLEASANT',
   
}

In [14]:
observations = [
    "cloud_cover", "global_radiation", "humidity", "precipitation",
    "pressure", "sunshine", "temp_max", "temp_mean", "temp_min"
]

In [15]:
# Create a copy of weather to avoid modifying original
weather_avg = pd.DataFrame()

for obs in observations:
    # Get all columns ending with the observation name
    cols = [col for col in weather.columns if col.endswith(obs)]
    weather_avg[obs] = weather[cols].mean(axis=1)


In [16]:
# Take the majority vote across all stations for each day
answers_binary = answers.apply(lambda col: col.map({'PLEASANT': 1, 'UNPLEASANT': 0}))
answers_binary['SONNBLICK_pleasant_weather'] = 0 #no pleasant weather recorded
y_raw = answers_binary.to_numpy()

In [17]:
def build_sequences(X_raw, y_raw, window_size=15):
  
    X_seq, y_seq = [], []

    for i in range(len(X_raw) - window_size + 1):
        # Features: 15-day window of observations
        X_window = X_raw[i:i + window_size]  # shape: (15, 9)

        # Labels: 15-day window across 15 stations → take majority vote per day
        y_window = y_raw[i:i + window_size]  # shape: (15, 15)
        y_majority = (np.mean(y_window, axis=1) > 0.5).astype(int)  # shape: (15,)

        # One-hot encode each day's label
        y_oh = np.eye(2)[y_majority]  # shape: (15, 2)

        # Append to sequences
        X_seq.append(X_window)
        y_seq.append(y_oh)

    return np.array(X_seq), np.array(y_seq)


In [18]:
# Extract NumPy arrays
X_raw = weather_avg.to_numpy()       # (22950, 9)
y_raw = answers_binary.to_numpy()  # (22950, 15)

# Build final sequences
X_seq, y_seq = build_sequences(X_raw, y_raw, window_size=15)

In [19]:
print("X_seq shape:", X_seq.shape)  # (22936, 15, 9)
print("y_seq shape:", y_seq.shape)  # (22936, 15, 2)

X_seq shape: (22936, 15, 9)
y_seq shape: (22936, 15, 2)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [21]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18348, 15, 9) (18348, 15, 2)
(4588, 15, 9) (4588, 15, 2)


In [22]:
print(X_train.shape, y_train.shape)

(18348, 15, 9) (18348, 15, 2)


In [23]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, InputLayer
from tensorflow.keras.layers import TimeDistributed

epochs = 30
batch_size = 16
n_hidden = 32

timesteps = 15
input_dim = 9
n_classes = 2

model = Sequential()
model.add(InputLayer(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=3, activation='relu', padding='same'))
model.add(TimeDistributed(Dense(16, activation='relu')))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(X_train,y_train, batch_size=16, epochs=30, verbose=2)

Epoch 1/30
1147/1147 - 4s - 4ms/step - accuracy: 0.9895 - loss: 0.0598
Epoch 2/30
1147/1147 - 3s - 2ms/step - accuracy: 1.0000 - loss: 5.6776e-07
Epoch 3/30
1147/1147 - 3s - 2ms/step - accuracy: 1.0000 - loss: 2.5350e-07
Epoch 4/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.7366e-07
Epoch 5/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.4342e-07
Epoch 6/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.3030e-07
Epoch 7/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2435e-07
Epoch 8/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2159e-07
Epoch 9/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2031e-07
Epoch 10/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1973e-07
Epoch 11/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1945e-07
Epoch 12/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1933e-07
Epoch 13/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1927e-07
Epoch 14/30
1147/1147 - 2

In [26]:
def confusion_matrix(Y_true, Y_pred):
    class_day = {0: 'UNPLEASANT', 1: 'PLEASANT'}
    y_true_labels = [class_day[y] for y in np.argmax(Y_true, axis=-1).flatten()]
    y_pred_labels = [class_day[y] for y in np.argmax(Y_pred, axis=-1).flatten()]
    return pd.crosstab(pd.Series(y_true_labels, name="Actual"),
                       pd.Series(y_pred_labels, name="Predicted"))

In [27]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Predicted   UNPLEASANT
Actual                
UNPLEASANT       68820


In [28]:
# Just for class count check (does NOT modify original y_train/y_test)
y_train_flat = np.argmax(y_train, axis=-1).flatten()
y_test_flat = np.argmax(y_test, axis=-1).flatten()

print("Train class distribution:")
print(pd.Series(y_train_flat).value_counts())

print("\nTest class distribution:")
print(pd.Series(y_test_flat).value_counts())


Train class distribution:
0    275220
Name: count, dtype: int64

Test class distribution:
0    68820
Name: count, dtype: int64


In [29]:
print(answers.apply(pd.Series.value_counts))

   BASEL_pleasant_weather  BELGRADE_pleasant_weather  \
0                   17286                      14958   
1                    5664                       7992   

   BUDAPEST_pleasant_weather  DEBILT_pleasant_weather  \
0                      15519                    18491   
1                       7431                     4459   

   DUSSELDORF_pleasant_weather  HEATHROW_pleasant_weather  \
0                        18016                      17991   
1                         4934                       4959   

   KASSEL_pleasant_weather  LJUBLJANA_pleasant_weather  \
0                    19176                       16574   
1                     3774                        6376   

   MAASTRICHT_pleasant_weather  MADRID_pleasant_weather  \
0                        18184                    12703   
1                         4766                    10247   

   MUNCHENB_pleasant_weather  OSLO_pleasant_weather  \
0                      18183                  19371   
1           

In [31]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = 15
input_dim = 9
n_classes = 2

mode = Sequential()
mode.add(InputLayer(shape=(timesteps, input_dim)))
mode.add(Conv1D(n_hidden, kernel_size=3, activation='relu', padding='same'))
mode.add(TimeDistributed(Dense(16, activation='relu')))
mode.add(TimeDistributed(Dense(n_classes, activation='softmax')))

In [32]:
mode.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
mode.fit(X_train,y_train, batch_size=16, epochs=30, verbose=2)

Epoch 1/30
1147/1147 - 13s - 12ms/step - accuracy: 0.9957 - loss: 0.0124
Epoch 2/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 5.6453e-07
Epoch 3/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 2.4294e-07
Epoch 4/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.6689e-07
Epoch 5/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.3951e-07
Epoch 6/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2808e-07
Epoch 7/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2307e-07
Epoch 8/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.2086e-07
Epoch 9/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1993e-07
Epoch 10/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1953e-07
Epoch 11/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1936e-07
Epoch 12/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1929e-07
Epoch 13/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1925e-07
Epoch 14/30
1147/1147 -

In [34]:
# Evaluate
print(confusion_matrix(y_test, mode.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Predicted   UNPLEASANT
Actual                
UNPLEASANT       68820


In [35]:
epochs = 30
batch_size = 16
n_hidden = 2

timesteps = 15
input_dim = 9
n_classes = 2

mod = Sequential()
mod.add(InputLayer(shape=(timesteps, input_dim)))
mod.add(Conv1D(n_hidden, kernel_size=3, activation='relu', padding='same'))
mod.add(TimeDistributed(Dense(16, activation='relu')))
mod.add(TimeDistributed(Dense(n_classes, activation='softmax')))

In [36]:
mod.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
mod.fit(X_train,y_train, batch_size=16, epochs=30, verbose=2)

Epoch 1/30
1147/1147 - 4s - 3ms/step - accuracy: 0.9916 - loss: 0.0275
Epoch 2/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.1197e-04
Epoch 3/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 2.7200e-05
Epoch 4/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 9.1820e-06
Epoch 5/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 3.7629e-06
Epoch 6/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.7539e-06
Epoch 7/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 9.0565e-07
Epoch 8/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 5.1813e-07
Epoch 9/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 3.2765e-07
Epoch 10/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 2.3128e-07
Epoch 11/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.8097e-07
Epoch 12/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.5367e-07
Epoch 13/30
1147/1147 - 2s - 2ms/step - accuracy: 1.0000 - loss: 1.3870e-07
Epoch 14/30
1147/1147 - 2

In [38]:
# Evaluate
print(confusion_matrix(y_test, mod.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Predicted   UNPLEASANT
Actual                
UNPLEASANT       68820
